In [621]:
import pandas as pd
import requests
import datetime
from math import cos, asin, sqrt
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [622]:
def get_parking_data():
    response = requests.get('https://opendata.arcgis.com/datasets/0060469c57864becb76a036d23236143_0.geojson').json()
    df = pd.io.json.json_normalize(response['features'])
    df = df.drop(df.columns[0], axis=1)
    new_columns = [col.rsplit('.')[1].upper() for col in df.columns.tolist()]
    df.columns = new_columns
    df[['LAT','LNG']] = pd.DataFrame(df['COORDINATES'].values.tolist(), index=df.index)
    return df
df = get_parking_data()

In [536]:
# haversine formula returns distance in km
# https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

# gets distance between two points
def distance(lat1, lng1, lat2, lng2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lng2-lng1)*p)) / 2
    return 12742 * asin(sqrt(a))

# returns the closest point
def closest(coords, v):   
    return min(coords, key=lambda p: distance(v[0],v[1],p[0],p[1]))

In [537]:
# returns date of next street sweeping datetime
def next_sweeping_date(clean_date): 
    if clean_date is None: 
        return None   
    digit = clean_date[0]    
    wkdy = time.strptime(clean_date.split()[-1][:3],"%a").tm_wday
    now = datetime.datetime.today()
    next_day = now.date() + datetime.timedelta(days=(wkdy-now.weekday())%7)    
    if not digit.isdigit() or next_day.day < 7: 
        return next_day
    next_month = pd.datetime(year=now.year, month=now.month + 1, day=1).date()
    if next_month.weekday() == wkdy: 
        return next_month
    return next_month.replace(day=(1 + (wkdy - next_month.weekday()) % 7))

In [538]:
# returns start and end datetimes of next street cleaning
def next_sweeping_period(clean_day, begin, end): 
    clean_date = next_sweeping_date(clean_day)
    if None in [clean_date, begin, end]: 
        return None
    start_datetime = datetime.datetime.combine(clean_date, datetime.datetime.strptime(begin, '%I %p').time())
    end_datetime = datetime.datetime.combine(clean_date, datetime.datetime.strptime(end, '%I %p').time())
    now = datetime.datetime.today()
    if now > end_datetime: 
        start_datetime += datetime.timedelta(days=7)
        end_datetime += datetime.timedelta(days=7)
    return [start_datetime, end_datetime]

In [539]:
# returns hours to next street sweeping
def time_to_sweeping(start_datetime): 
    td = start_datetime - datetime.datetime.today()
    if td < datetime.timedelta(hours=0): 
        return 0
    return td // pd.Timedelta(hours=1)

In [540]:
# returns True for timelimit violations
def timelimit_violation(limit): 
    if limit in ['No Parking Anytime', '30 Minutes', '5 Minutes', '1 Hour', '15 Minutes', '90 Minutes']: 
        return True
    return False

In [541]:
# returns True for parking type violations
def pkgtype_violation(pkgtype): 
    if pkgtype in ['Red Zone', 'Single Space Meter', 'Red Zone (Fire Hydrant)', 'Alley', 'White Zone',
                   'No Parking Any Time', 'Yellow Zone', 'Fire Hydrant', 'Crosswalk', 'Residential Zone',
                   'Single Space Meter (Double Space)', 'Crosswalk (No Markings)', 'No Parking Passenger Loading', 
                   'Motorcycle Parking', 'R X R', 'Blue Zone', 'Cross Hatching', 'Taxi Zone', 'Bus Only', 
                   'Official Vehicle Only', 'Construction', 'Fire Lane', 'Passenger Loading Zone', 'Bike Parking']: 
        return True
    return False

In [552]:
# returns a dictionary for the coordinate
def get_parking(coord):
    coord_dict = df[(df['LAT'] == coord[0]) & (df['LNG'] == coord[1])].to_dict(orient='records')[0]
    coord_dict['PARKING_VIOLATION'] = timelimit_violation(coord_dict['TIMELIMIT']) or pkgtype_violation(coord_dict['PKGTYPE'])
    next_sweep = next_sweeping_period(coord_dict['PKGSDAY'], coord_dict['PKGSWBEG'], coord_dict['PKGSWEND'])    
    if next_sweep is None: 
        for key in ['NEXT_SWEEPING_START', 'NEXT_SWEEPING_END', 'NEXT_SWEEPING_HRS']:
            coord_dict[key] = None
    else: 
        coord_dict['NEXT_SWEEPING_START'] = next_sweep[0].strftime("%Y-%m-%d %H:%M:%S")
        coord_dict['NEXT_SWEEPING_END'] = next_sweep[1].strftime("%Y-%m-%d %H:%M:%S")
        coord_dict['NEXT_SWEEPING_HRS'] = time_to_sweeping(next_sweep[0])
    return coord_dict

In [610]:
class ParkingSpot:
    def __init__(self, coord):
        self.originalCoord = coord        
        self.coordinates = closest(df['COORDINATES'].tolist(), coord)
        self.all = get_parking(self.coordinates)
        self.coordinates = self.all['COORDINATES']
        self.distM = distance(coord[0], coord[1], self.coordinates[0], self.coordinates[1]) * 1000
        self.fullAddress = f"{self.all['ADDRESS']} {self.all['STREET']}{self.all['SUFFIX']} {self.all['PREFIX']}"
        self.timeLimit = self.all['TIMELIMIT']
        self.parkingType = self.all['PKGTYPE']
        self.permitArea = self.all['PERMITAREA']
        self.eventArea = self.all['EVTAREA']
        self.violation = self.all['PARKING_VIOLATION']
        self.sweepStart = self.all['NEXT_SWEEPING_START']
        self.sweepEnd = self.all['NEXT_SWEEPING_END']
        self.sweepHrs = self.all['NEXT_SWEEPING_HRS']
        
current = ParkingSpot([-121.50, 38.5740883151661])
print(current.distM)

28.9200191056914


In [567]:
print(get_parking([-121.50350012231698, 38.57408831516617]))

{'GISOBJID': 10014609, 'OBJ_CODE': 'PKGS990', 'ADDRESS': '1707', 'AORB': None, 'STREET': '5', 'SUFFIX': 'TH', 'PREFIX': 'ST', 'EVENODD': 'ODD', 'TIMELIMIT': 'No Parking Anytime', 'PKGTYPE': 'Red Zone', 'AORP': 'Parallel', 'PERMITAREA': None, 'MAXRATE': nan, 'EVTAREA': 'C', 'PKGENDAY': None, 'ENBEGIN': None, 'ENEND': None, 'PKGSDAY': None, 'PKGSWBEG': None, 'PKGSWEND': None, 'PARKMOB': None, 'TMSTRCN': None, 'NOPARKDAYS': None, 'NOPARKTIME': None, 'BEAT_NUM': 9.0, 'ASSET_ID': 990, 'OBJECTID': 1005, 'TYPE': 'Point', 'COORDINATES': [-121.50350012231698, 38.57408831516617], 'LAT': -121.50350012231698, 'LNG': 38.57408831516617, 'PARKING_VIOLATION': True, 'NEXT_SWEEPING_START': None, 'NEXT_SWEEPING_END': None, 'NEXT_SWEEPING_HRS': None}


In [582]:
df['NOPARKDAYS'].unique()

array([None, 'Mon-Sat', 'Mon-Fri', 'Fri-Sun', 'Sun-Sat',
       'Mon-Sat & Fri-Sun', 'Sat-Sun'], dtype=object)

In [620]:
df.head()

,GISOBJID,OBJ_CODE,ADDRESS,AORB,STREET,SUFFIX,PREFIX,EVENODD,TIMELIMIT,PKGTYPE,AORP,PERMITAREA,MAXRATE,EVTAREA,PKGENDAY,ENBEGIN,ENEND,PKGSDAY,PKGSWBEG,PKGSWEND,PARKMOB,TMSTRCN,NOPARKDAYS,NOPARKTIME,BEAT_NUM,ASSET_ID,OBJECTID,TYPE,COORDINATES,LAT,LNG
0,10014609,PKGS990,1707,None,5,TH,ST,ODD,No Parking Anytime,Red Zone,Parallel,None,NaN,C,None,None,None,None,None,None,None,None,None,None,9.0,990,1005,Point,"[-121.50350012231698, 38.57408831516617]",-121.503500,38.574088
1,10014610,PKGS991,1708,None,5,TH,ST,EVEN,2+,Single Space Meter,Parallel,None,NaN,C,MON-SAT,8 am,10 pm,None,None,None,No,None,None,None,9.0,991,1006,Point,"[-121.50368758285819, 38.57403925262942]",-121.503688,38.574039
2,10014611,PKGS992,1709,None,5,TH,ST,ODD,No Parking Anytime,RT,Parallel,None,NaN,C,None,None,None,None,None,None,None,None,None,None,9.0,992,1007,Point,"[-121.50352632905881, 38.574027624292384]",-121.503526,38.574028
3,10014612,PKGS993,1710,None,5,TH,ST,EVEN,2+,Single Space Meter,Parallel,None,NaN,C,MON-SAT,8 am,10 pm,None,None,None,No,None,None,None,9.0,993,1008,Point,"[-121.50371378941621, 38.573978562614066]",-121.503714,38.573979
4,10014613,PKGS994,1711,None,5,TH,ST,ODD,No Parking Anytime,Driveway,Parallel,None,NaN,C,None,None,None,None,None,None,None,None,None,None,9.0,994,1009,Point,"[-121.50355253689732, 38.57396693431839]",-121.503553,38.573967
